In [1]:
import requests
import json
import prettytable
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file_metro = "data_files\MSA_M2018_dl.xlsx"
bls_metro_may2018_df = pd.read_excel(file_metro)
bls_metro_may2018_df.head()

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#,NaN,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140,True,NaN


In [4]:
bls_metro_may2018_df.nunique()

PRIM_STATE         52
AREA              395
AREA_NAME         395
OCC_CODE          825
OCC_TITLE         825
OCC_GROUP           3
TOT_EMP          3852
EMP_PRSE          501
JOBS_1000       21684
LOC QUOTIENT     1553
H_MEAN           7881
A_MEAN          13846
MEAN_PRSE         297
H_PCT10          4843
H_PCT25          5799
H_MEDIAN         6701
H_PCT75          7760
H_PCT90          8708
A_PCT10          8490
A_PCT25         10225
A_MEDIAN        12239
A_PCT75         14635
A_PCT90         16796
ANNUAL              1
HOURLY              1
dtype: int64

In [5]:
bls_metro_2018_df = bls_metro_may2018_df.drop([
                                    'ANNUAL',
                                    'HOURLY'] ,axis=1)
bls_metro_2018_df.head()

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140


In [6]:
x = bls_metro_2018_df.columns.to_list()
x

['PRIM_STATE',
 'AREA',
 'AREA_NAME',
 'OCC_CODE',
 'OCC_TITLE',
 'OCC_GROUP',
 'TOT_EMP',
 'EMP_PRSE',
 'JOBS_1000',
 'LOC QUOTIENT',
 'H_MEAN',
 'A_MEAN',
 'MEAN_PRSE',
 'H_PCT10',
 'H_PCT25',
 'H_MEDIAN',
 'H_PCT75',
 'H_PCT90',
 'A_PCT10',
 'A_PCT25',
 'A_MEDIAN',
 'A_PCT75',
 'A_PCT90']

In [7]:
# bls_metro_may2018_df = bls_metro_may2018_df.drop(['AREA_TYPE',
#                                     'NAICS',
#                                     'NAICS_TITLE',
#                                     'I_GROUP',
#                                     'OWN_CODE',
#                                     'PCT_TOTAL',
#                                     'ANNUAL',
#                                     'HOURLY'] ,axis=1)
# bls_metro_may2018_df.head()

In [8]:
x = [col.lower() for col in x]
x

['prim_state',
 'area',
 'area_name',
 'occ_code',
 'occ_title',
 'occ_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

In [9]:
bls_metro_2018_df.columns = x

In [10]:
bls_metro_2018_df.head()

,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140


In [11]:
x

['prim_state',
 'area',
 'area_name',
 'occ_code',
 'occ_title',
 'occ_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

In [12]:
cols = ['tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

for col in cols:
    bls_metro_2018_df[col] = pd.to_numeric(bls_metro_2018_df[col], errors='coerce')

bls_metro_2018_df.head()

,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400.0,8.7,2.307,1.71,89.00,185120.0,4.4,41.22,57.53,78.04,NaN,NaN,85730.0,119660.0,162320.0,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630.0,4.2,21.157,1.34,59.23,123200.0,2.7,26.53,35.99,50.10,71.35,NaN,55190.0,74860.0,104210.0,148400.0,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50.0,1.3,0.293,0.84,NaN,41580.0,15.0,NaN,NaN,NaN,NaN,NaN,20690.0,27640.0,51130.0,51140.0,51140.0


In [13]:
bls_metro_may2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143905 entries, 0 to 143904
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   PRIM_STATE    143905 non-null  object
 1   AREA          143905 non-null  int64 
 2   AREA_NAME     143905 non-null  object
 3   OCC_CODE      143905 non-null  object
 4   OCC_TITLE     143905 non-null  object
 5   OCC_GROUP     143905 non-null  object
 6   TOT_EMP       143905 non-null  object
 7   EMP_PRSE      143905 non-null  object
 8   JOBS_1000     143905 non-null  object
 9   LOC QUOTIENT  143905 non-null  object
 10  H_MEAN        143905 non-null  object
 11  A_MEAN        143905 non-null  object
 12  MEAN_PRSE     143905 non-null  object
 13  H_PCT10       143905 non-null  object
 14  H_PCT25       143905 non-null  object
 15  H_MEDIAN      143905 non-null  object
 16  H_PCT75       143905 non-null  object
 17  H_PCT90       143905 non-null  object
 18  A_PCT10       143905 non

In [14]:
bls_metro_2018_df = bls_metro_2018_df.dropna()

In [15]:
bls_metro_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118961 entries, 0 to 143904
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   prim_state    118961 non-null  object 
 1   area          118961 non-null  int64  
 2   area_name     118961 non-null  object 
 3   occ_code      118961 non-null  object 
 4   occ_title     118961 non-null  object 
 5   occ_group     118961 non-null  object 
 6   tot_emp       118961 non-null  float64
 7   emp_prse      118961 non-null  float64
 8   jobs_1000     118961 non-null  float64
 9   loc quotient  118961 non-null  float64
 10  h_mean        118961 non-null  float64
 11  a_mean        118961 non-null  float64
 12  mean_prse     118961 non-null  float64
 13  h_pct10       118961 non-null  float64
 14  h_pct25       118961 non-null  float64
 15  h_median      118961 non-null  float64
 16  h_pct75       118961 non-null  float64
 17  h_pct90       118961 non-null  float64
 18  a_pc

In [16]:
bls_metro_2018_df.head(5)

,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0
5,AK,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,detailed,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0
6,AK,11260,"Anchorage, AK",11-2021,Marketing Managers,detailed,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0
8,AK,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,detailed,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0


In [17]:
bls_metro_2018_df["Date"]="May 2018"

In [18]:
bls_metro_2018_df.head(5)

,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018
5,AK,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,detailed,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018
6,AK,11260,"Anchorage, AK",11-2021,Marketing Managers,detailed,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018
8,AK,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,detailed,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018


In [19]:
bls_metro_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118961 entries, 0 to 143904
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   prim_state    118961 non-null  object 
 1   area          118961 non-null  int64  
 2   area_name     118961 non-null  object 
 3   occ_code      118961 non-null  object 
 4   occ_title     118961 non-null  object 
 5   occ_group     118961 non-null  object 
 6   tot_emp       118961 non-null  float64
 7   emp_prse      118961 non-null  float64
 8   jobs_1000     118961 non-null  float64
 9   loc quotient  118961 non-null  float64
 10  h_mean        118961 non-null  float64
 11  a_mean        118961 non-null  float64
 12  mean_prse     118961 non-null  float64
 13  h_pct10       118961 non-null  float64
 14  h_pct25       118961 non-null  float64
 15  h_median      118961 non-null  float64
 16  h_pct75       118961 non-null  float64
 17  h_pct90       118961 non-null  float64
 18  a_pc

In [20]:
#create CSV files to be used to load database as needed
bls_metro_2018_df.to_csv('data_files/bls_metro_2018_clean.csv')
